In [1]:
!pip install allennlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install allennlp-models

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install cached-path==1.1.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached cached_path-1.1.2-py3-none-any.whl (26 kB)
  Using cached huggingface_hub-0.5.1-py3-none-any.whl (77 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.8.1
    Uninstalling huggingface-hub-0.8.1:
      Successfully uninstalled huggingface-hub-0.8.1
  Attempting uninstall: cached-path
    Found existing installation: cached-path 1.1.5
    Uninstalling cached-path-1.1.5:
      Successfully uninstalled cached-path-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.10.0 requires cached-path<1.2.0,>=1.1.3, but you have cached-path 1.1.2 which is incompatible.


In [4]:
!pip install --upgrade google-cloud-storage

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached spacy-3.4.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.3 MB)
  Using cached thinc-8.1.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (821 kB)
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.17
    Uninstalling thinc-8.0.17:
      Successfully uninstalled thinc-8.0.17
  Attempting uninstall: spacy
    Found existing installation: spacy 3.3.1
    Uninstalling spacy-3.3.1:
      Successfully uninstalled spacy-3.3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.10.0 requires cached-path<1.2.0,>=1.1.3, but you have cached-path 1.1.2 which is incompatible.
allennlp 2.10.0 requires spacy<3.4,>=2.1.0, but you have spacy 3.4.1 which is incompatible.


In [6]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
%matplotlib inline
import tensorflow as tf
import torch
import os
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from sklearn.multiclass import OneVsRestClassifier
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, random_split
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
import numpy as np
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertTokenizer
from transformers import get_linear_schedule_with_warmup
import time
import datetime
from sklearn.model_selection import train_test_split
import random
import re
import matplotlib
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertModel, BertPreTrainedModel
from transformers.modeling_outputs import SequenceClassifierOutput
from torch import nn
from torch.nn import BCEWithLogitsLoss
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import spacy
import pandas as pd
from tqdm import tqdm

In [8]:
import nltk
nltk.download("stopwords")
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
import nltk
nltk.download('universal_tagset')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

The first experiment proceeds on the word level and jokes with repetitive words in the first and last sentences were extracted from the corpus. The amount of working data reduces from 1000 to 128 texts. The selected humorous texts are the inputs of the standard neural model BERT, which creates word embedding based on  its context. The cosine sinilarity is calculated between vectors of two different positions of the same word.

In [16]:
from nltk import word_tokenize



dt = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR5otRvtehsdCl9OhmxpkyI35isTuIuEAL7Golcrd9SegpK8nv8GINj6hkABG7YyGr5O5KQmA-pPCga/pub?gid=1658698786&single=true&output=csv")
dt["joke"] = [i.lower() for i in dt["joke"]]
dt = dt[dt["correct_polarity"] != None]
stop_words = []
pos_tagger =[]
good_jokes = []

for item in tqdm(dt.to_dict(orient="records"), total = len(dt.to_dict(orient="records"))):
  for word in word_tokenize(item["punchline"]):
    word = word.replace('"', '')
    word_tag = nltk.pos_tag([word])[0][1]
    if word in  (stopwords.words('english') + ["for", '', '.', '?', '!', ';', ':', '“', '”']):
      continue
    elif word_tag != 'NN':
      continue
    else:
      count_word_setup = item["setup"].count(word)
      count_word_punch = item["punchline"].count(word)
      if count_word_setup > 1:
        good_jokes.append({"joke": item["joke"], "word": word, "word_tag": word_tag, "word_count_setup": count_word_setup, "word_count_punchline":count_word_punch})
      else:
        pass

100%|██████████| 1088/1088 [00:02<00:00, 386.37it/s]


In [17]:
jokes_repeat_words = pd.DataFrame(good_jokes)
jokes_repeat_words = jokes_repeat_words.drop_duplicates()
jokes_repeat_words = jokes_repeat_words.groupby(["joke"]).agg({"word": list, "word_tag": list, "word_count_setup": list}).reset_index().reindex()
jokes_repeat_words["id_joke"] = [i for i in range(0, len(jokes_repeat_words))]
jokes_repeat_words[jokes_repeat_words["word"].str.len() > 0]

,joke,word,word_tag,word_count_setup,id_joke
0,a bus station is where a bus stops . a train s...,[station],[NN],[2],0
1,a city feller goes to a dude ranch and arrives...,[city],[NN],[2],1
2,a couple in their sixties are walking along th...,"[genie, want, wife, *]","[NN, NN, NN, NN]","[4, 2, 4, 2]",2
3,"a cross and hungry panda went into a bar and, ...","[end, bar, panda]","[NN, NN, NN]","[2, 4, 2]",3
4,a dog called rufus jagger walks into a bank . ...,"[dog, man]","[NN, NN]","[2, 3]",4
...,...,...,...,...,...
100,two owls are playing in the final of the owl p...,[owl],[NN],[3],100
101,two robbers went the gates of heaven an angel ...,[god],[NN],[2],101
102,what is red and looks like a bucket ? a red b...,[bucket],[NN],[3],102
103,why did the squirrel fall out of the tree ? i...,[squirrel],[NN],[2],103


In [18]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
v = tokenizer.get_vocab()
print(len(v))
#tokenizer.add_tokens(['barman', 'amahl', 'giraffe', 'outrun', 'physchiatrist', 'hollandaise', 'wack', 'breeder', 'ostrich']) # если добавлять слова в tokenizer, потом ломается модель


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

30522


In [19]:
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [20]:
jokes_embed = []
for thing in tqdm(jokes_repeat_words.to_dict(orient ="records"), total = len(jokes_repeat_words)):
    marked_text = "[CLS] " + thing["joke"] + " [SEP]"

    # Split the sentence into tokens.
    tokenized_text = tokenizer.tokenize(marked_text)

    # Map the token strings to their vocabulary indeces.
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

    # Display the words with their indeces.
    #for tup in zip(tokenized_text, indexed_tokens):
    #    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

    segments_ids = [1] * len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    with torch.no_grad():

        outputs = model(tokens_tensor, segments_tensors)

        # Evaluating the model will return a different number of objects based on 
        # how it's  configured in the `from_pretrained` call earlier. In this case, 
        # becase we set `output_hidden_states = True`, the third item will be the 
        # hidden states from all layers. See the documentation for more details:
        # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
        hidden_states = outputs[2]

    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)

    token_embeddings.size()
    # Stores the token vectors, with shape [22 x 768]
    token_vecs_sum = []

    # `token_embeddings` is a [22 x 12 x 768] tensor.

    # For each token in the sentence...
    for token in token_embeddings:

        # `token` is a [12 x 768] tensor

        # Sum the vectors from the last four layers.
        sum_vec = torch.sum(token[-4:], dim=0)
        
        # Use `sum_vec` to represent `token`.
        token_vecs_sum.append(sum_vec)

    jokes_embed.append({"id_joke": thing["id_joke"], "joke": thing["joke"], "tokenized_text": tokenized_text, "token_vecs_sum": token_vecs_sum})

#print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

100%|██████████| 105/105 [00:42<00:00,  2.46it/s]


In [21]:
words_index = []
for thing, embed in zip(jokes_repeat_words.to_dict(orient="records"),jokes_embed):
  for word in thing['word']:
    word_positions = []
    for i, item in enumerate(embed["tokenized_text"]):
      if item == word:
        word_positions.append(i)
      else:
        continue
    words_index.append({"id_joke": thing["id_joke"], 'joke': "CLS " + thing["joke"] + " SEP","word": word, 'word_positions': word_positions})

In [22]:
from scipy.spatial.distance import cosine


results = []
for item in tqdm(words_index, total = len(words_index)):
  if len(item["word_positions"]) > 0:
    sim = 1 - cosine(jokes_embed[item["id_joke"]]["token_vecs_sum"][item["word_positions"][0]], jokes_embed[item["id_joke"]]["token_vecs_sum"][item["word_positions"][-1]])
    results.append({"cosine_sim": round(sim, 1), "word": item["word"], "word_positions": [item["word_positions"][0], item["word_positions"][-1]], "joke": item["joke"], "id_joke": item["id_joke"]})
  else:
    continue

100%|██████████| 140/140 [00:00<00:00, 7914.10it/s]


In [23]:
res_data = pd.DataFrame(results)

In [24]:
res_data

,cosine_sim,word,word_positions,joke,id_joke
0,0.8,station,"[3, 26]",CLS a bus station is where a bus stops . a tra...,0
1,0.8,city,"[2, 79]",CLS a city feller goes to a dude ranch and arr...,1
2,0.8,genie,"[39, 147]",CLS a couple in their sixties are walking alon...,2
3,0.6,want,"[83, 150]",CLS a couple in their sixties are walking alon...,2
4,0.6,wife,"[17, 152]",CLS a couple in their sixties are walking alon...,2
...,...,...,...,...,...
124,0.7,owl,"[10, 49]",CLS two owls are playing in the final of the o...,100
125,0.8,god,"[12, 51]",CLS two robbers went the gates of heaven an an...,101
126,0.8,bucket,"[8, 25]",CLS what is red and looks like a bucket ? a r...,102
127,0.8,squirrel,"[4, 45]",CLS why did the squirrel fall out of the tree ...,103


The results show that 90% of cases dones’t reflect the semantic shift, because it is not based on the repetition of this word, so understanding a joke expects more precise work with contexts. The cases, when the repeated word is associated with semantic shift, are about 3%. The rest percents compound jokes, the frames’ incongruity is tied to the other repetitive phrases or synonyms.


Second Experiment.

With the help of coreference links it is important to collect as many jokes with synonyms and pronouns as possible to find the semantic shift.

 That is why in the second experiment a working pipeline was completed with resolving of coreference – a type of textual or syntactic coherence in which two or more nominal groups describe the same object. We used the Span-BERT neural network from Python library allennlp for the recognition of antecedents and its anaphores. The cosine similarity is measured separately for each concept: the antecedent vector is compared to all embeddings of its anaphores. 
 

In [25]:
from allennlp.predictors.predictor import Predictor
import pandas as pd
from tqdm import tqdm
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2021.03.10.tar.gz")


downloading: 100%|##########| 1.25G/1.25G [00:42<00:00, 31.8MiB/s]


Downloading:   0%|          | 0.00/414 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/634M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-large-cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
def preproc(text):
  text = text.replace("'re", ' re ')
  text = text.replace('1,50', 'one and half')
  text = text.replace("won't", "will not")
  text = text.replace("don't", "do not ")
  text = text.replace("n't", " not ")
  text = text.replace("?", " ? ")
  text = text.replace(".", " . ")
  text = text.replace("!", " ! ")
  text = text.replace("[", " [ ")
  text = text.replace("]", " ] ")
  text = text.replace(":", " : ")
  text = text.replace(";", " ; ")
  text = text.replace(",", " , ")
  text = text.replace("'ll", " will")
  text = text.replace("'", "")
  text = text.replace('"', '')
  text = text.replace('-', '')
  text = text.replace("`", "")
  text = text.replace('…', '')
  text = text.replace('“', '') 
  text = text.replace('’', '')
  text = text.replace("  ", " ") 
  return text


#nlp = spacy.load("en_core_web_sm")
punkt = ["?", ',', '.', ':', '!', ';',  '"']

dt = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vQfFx3TMpHYWKXAiExoDC4qFpYal1n_y8yLAfQTHpUOCVFw67uukmf5rFd60McXEM0L6-vilYw2UK_q/pub?gid=0&single=true&output=csv")
dt = dt.drop_duplicates(["joke"]).reset_index(drop=True).reindex()
dt["joke"]= [preproc(text = item) for item in dt["joke"]] 
dt["id_joke"] = [i for i in range(0, len(set(dt['joke'])))]

In [35]:
jokes_embed = []
sent = "is it true that cannibals do not eat clowns because they taste funny?"
#for thing in tqdm(dt.to_dict(orient="records"), total = len(dt)):
marked_text = "[CLS] " + sent  + " [SEP]" #joke

# Split the sentence into tokens.
tokenized_text = tokenizer.tokenize(marked_text)
encodings = tokenizer(marked_text)

# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Display the words with their indeces.
#for tup in zip(tokenized_text, indexed_tokens):
#    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

segments_ids = [1] * len(tokenized_text)
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

with torch.no_grad():

    outputs = model(tokens_tensor, segments_tensors)

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]

token_embeddings = torch.stack(hidden_states, dim=0)
token_embeddings = torch.squeeze(token_embeddings, dim=1)
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()
# Stores the token vectors, with shape [22 x 768]
token_vecs_sum = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)


jokes_embed.append({"id_joke": thing["id_joke"], "joke": thing["joke"], "tokenized_text": tokenized_text, "token_vecs_sum": token_vecs_sum}) #"id_joke": thing["id_joke"]

In [30]:

nlp = spacy.load("en_core_web_sm")
all_coref = []
for joke in tqdm(jokes_embed, total = len(jokes_embed)):
  #print(joke["id_joke"])
  missed_jokes_coref = []
  sent = joke["joke"].lower()
  sent = tokenizer.tokenize(sent)
  pretok_sent = ""
  for tok in sent:
      if tok.startswith("##"):
          pretok_sent += tok[2:]
      else:
          pretok_sent += " " + tok
  pretok_sent = pretok_sent[1:]
  words_indexes = {i: word for i, word in enumerate(pretok_sent.split(' '))}


  pred = predictor.predict_tokenized(tokenized_document=pretok_sent.split(' '))

  clusters = pred['clusters']
  document = pred['document'] 

  n = 0
  docum = {}
  for obj in document:    
      docum.update({n :  obj}) #what I'm doing here is creating a dictionary of each word with its respective index, making it easier later.    
      n = n+1

  clus_all = []
  cluster = []
  clus_one = {}
  for i in range(0, len(clusters)):    
      one_cl = clusters[i]    
      for count in range(0, len(one_cl)):           
          obj = one_cl[count]        
          for num in range((obj[0]), (obj[1]+1)):            
              for n in docum:                
                  if num == n:                 
                      cluster.append(docum[n]) 
      clus_all.append(cluster)       
      cluster = [] 

  print(len(joke["tokenized_words_vecs"]), len(pretok_sent.split(' ')))


  one_clust = []
  for clust in clusters:
    one_clust = []
    for item in clust:
      string = []
      inds = []
      vectors = []
      if item[0] == item[1]:
        one_clust.append({'item':docum[item[0]],"index_item": [item[0]], "item_vector": joke["tokenized_words_vecs"][int(item[0])]["vector"]})
      elif item[0] != item[1]:
          string = []
          inds = []
          vectors = []
          i = item[0]
          while i <= item[1]:
            string.append(docum[i])
            inds.append(i)
            vectors.append(joke["tokenized_words_vecs"][i]["vector"])
            i += 1
          #print({'item':" ".join(string).replace('#', ''), 'index_item':inds})
          one_clust.append({'item':" ".join(string).replace('#', ''), 'index_item':inds,  "item_vector":sum(vectors)/len(vectors)})
          string = []
          inds = []
          vectors = []
    all_coref.append({"cluster_words": one_clust,  "joke": joke["joke"], "id_joke": joke["id_joke"], "words_indexes": words_indexes})
      
  doc = nlp(" ".join(pretok_sent.split(' ')))

  child_chunks = []
  for token in doc:
    for child in token.children:
      if token.text not in punkt and child.text not in punkt:
        child_chunks.append({"index_child":child.i, token.text:child.text})
      else:
        continue

  for clust_it in all_coref:
    for it in clust_it["cluster_words"]:
      if len(doc) == len(pretok_sent.split(' ')):
        print(it["index_item"], it["item"])
        it["head_word"] = doc[it["index_item"][-1]].head.text # Доп идея: БРАТЬ НЕ ПРЯМ САМ ГЛАГОЛ, А ОТ НЕГО ЗАВИСИМОЕ СКАЗУЕМОЕ, ЕСЛИ ОНО У НЕГО ЕСТЬ, чтобы сразу выходить на нужных детей.
        it["index_head_word"]  = [doc[it["index_item"][-1]].head.i]
        it["head_word_vector"] = joke["tokenized_words_vecs"][doc[it["index_item"][-1]].head.i]["vector"]
        child_dependencies =[]
        for child in doc[doc[it["index_item"][-1]].head.i].children: # дети от хэд-слова для кореференции. Для самой кореференции синтаксические дети не нужны, т.к. зачастую они уже выделены в кореференции
          if child.text not in punkt:
            childs = {"child":child.text, "index_child": child.i} # дети от хэд-слова для кореференции.
            for dep in doc[child.i].children:
              if dep.text not in punkt:
                child_dependencies.append(dep.i)  # далее идет максимальное извлечение всех возможных зависимых слов от детей 
            for ch in doc[child.i].rights: # rights - дочерние элементы слова справа от него в синтаксическом анализе зависимостей. Справа, т.к. идем в панчлайн.
              if ch.text not in punkt:
                for c in ch.children:
                    if c.text not in punkt:
                      child_dependencies.append(c.i)
                      for depd in doc[c.i].rights: 
                        if depd.text not in punkt:
                          for depde in depd.children:
                            child_dependencies.append(depde.i)
        child_dependencies_l = list(set([childs["index_child"]] + child_dependencies)) #удаляем из списка повторы + кореференции от зависимого слова
        child_dependencies_comp = sorted(list(set( i for i in child_dependencies_l if i not in it["index_item"])))
        try:
          child_dependencies = [{"child_vector":joke["tokenized_words_vecs"][i]["vector"], "child_index": i}  for i in child_dependencies_comp]
        except IndexError as ie:
          continue
        it["children"] = child_dependencies 
      #print(it["item"], doc[it["index_item"][-1]].head.text, it["children"])
  #except IndexError as ind:
    #print(joke["id_joke"])
    #continue

  0%|          | 1/1099 [00:01<20:17,  1.11s/it]

14 14
[8] clowns
[10] they


  0%|          | 2/1099 [00:01<13:59,  1.31it/s]

15 15
[8] clowns
[10] they


  0%|          | 3/1099 [00:02<11:09,  1.64it/s]

...

  0%|          | 5/1099 [00:06<24:50,  1.36s/it]

41 41


  0%|          | 5/1099 [00:07<26:44,  1.47s/it]

44 44
[8] clowns
[10] they
[0, 1, 2, 3, 4, 5] a man walking down the streets
[15, 16] one man
[35, 36] the man
[42] his
[57, 58, 59] the injured man


IndexError: ignored

In [ ]:
#### главное слово сравнивается с предпоследним и последним. Всего три пары: первое - предпоследнее; предпоследнее - последнее; первое - последнее.
from scipy.spatial.distance import cosine



cos = torch.nn.CosineSimilarity(dim=0)
result = []
for dikt in tqdm(all_coref, total = len(all_coref)):
  if len(dikt['cluster_words']) > 2:
    cosin_sim_1_2 = cos(dikt['cluster_words'][0]["word_vector"], dikt['cluster_words'][len(dikt['cluster_words']) - 2]['word_vector']) 
    cosin_sim_2_3 = cos(dikt['cluster_words'][len(dikt['cluster_words']) - 2]["word_vector"], dikt['cluster_words'][len(dikt['cluster_words'])-1]['word_vector'])
    cosin_sim_1_3 = cos(dikt['cluster_words'][0]["word_vector"], dikt['cluster_words'][len(dikt['cluster_words'])- 1]['word_vector'])
    result_dikt = {"cluster_main_first_word": dikt['cluster_words'][0]['word'], 
                    "cluster_prelast_second_word": dikt['cluster_words'][len(dikt['cluster_words']) - 2]['word'],
                    "cluster_last_third_word": dikt['cluster_words'][len(dikt['cluster_words'])-1]['word'],
                  "cosine_simil_first_sec":cosin_sim_1_2, "cosine_simil_sec_third":cosin_sim_2_3, "cosine_simil_first_third": cosin_sim_1_3,
                  "min_cosine_sim": min([cosin_sim_1_2, cosin_sim_1_3, cosin_sim_2_3]), "max_cosine_sim": max([cosin_sim_1_2, cosin_sim_1_3, cosin_sim_2_3]), 
                    "joke_tokenized": dikt["joke"], "id_joke": dikt["id_joke"]}
  elif len(dikt['cluster_words']) <= 2:
    cosin_sim_1_2 = cos(dikt['cluster_words'][0]["word_vector"], dikt['cluster_words'][len(dikt['cluster_words']) - 1]['word_vector'])
    result_dikt = {"cluster_main_first_word": dikt['cluster_words'][0]['word'], 
                  "cluster_prelast_second_word": dikt['cluster_words'][len(dikt['cluster_words']) - 1]['word'],
                  "cluster_last_third_word": '-',
                "cosine_simil_first_sec":cosin_sim_1_2, "cosine_simil_sec_third": 0, "cosine_simil_first_third": 0,
                 "min_cosine_sim": 0, "max_cosine_sim": 0, 
                  "joke_tokenized": dikt["joke"], "id_joke": dikt["id_joke"]}
  result.append(result_dikt)



In [ ]:
df = pd.DataFrame(result)

In [ ]:
df.to_csv("jokes_expreiment_one_coreference_shift_version_sec.csv", encoding='utf-8')

In [ ]:
list_one_two = []
list_one_three = []
list_two_three = []
dt = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR5otRvtehsdCl9OhmxpkyI35isTuIuEAL7Golcrd9SegpK8nv8GINj6hkABG7YyGr5O5KQmA-pPCga/pub?gid=214916818&single=true&output=csv")
dt = dt[dt["min_cosine_sim"] != "0"]
for i in dt.to_dict(orient= "records"):
  if i["min_cosine_sim"] == i["cosine_simil_first_sec"]:
    list_one_two.append(i["cosine_simil_first_sec"])
  elif i["min_cosine_sim"] == i["cosine_simil_sec_third"]:
    list_two_three.append(i["cosine_simil_sec_third"])
  elif i["min_cosine_sim"] == i["cosine_simil_first_third"]:
    list_one_three.append(i["cosine_simil_first_third"])
print("Кол-во шуток, где минимальный косинус между разными парами слов в кластерах кореференций ", len(list_one_two), len(list_two_three), len(list_one_three))

После получения сущностей (главных слов в кластере кореференций) и связанных с ними слов (кореференции), а также косинусного расстояния между ними, был проведен качественный анализ результатов. При разборе шуток отмечались следующие моменты:

1) связана ли пара "главная сущность- его кореференция" с семантическим сдвигом. Если да, то насколько. Было выделено два случая.

- связь прямая: "главная сущность- его кореференция" целиком затрагивают семантический сдвиг, дополнительная предобработка не требуется. Сдвиг сразу можно отследить. 

- связь косвенная: "главная сущность- его кореференция" частично затрагивают семантический сдвиг, то есть не хватает еще слова или выражения, чтобы связь сразу можно было бы назвать явной. Обычно вместе с этим указывается, что нужен синтаксический разбор и надо сопоставлять куски текста, чтобы точно получить сдвиг.

2) Если среди пар "главная сущность- его кореференция" не находилось таких, где отражался бы семантический сдвиг, то шутке приписывалось "хотя слова были выделены правильно, надо сопоставлять куски текста" и зачастую также добавлялось, что ""нужен синтаксический разбор", которые помог бы скорректировать выявлений сущностей и кореференций.

3) Встречались случаи, когда пары "главная сущность- его кореференция" были составлены автоматически неправильно, либо же были недовыделены зависимые сущности у главного слова. Такие шутки были отнесены в категорию "проблемы в предобработке". Поскольку на вход Span-Bert поступали уже токенизированные бертом шутки, где иногда встречались сабворды, возникло предположение, что проблема в плохой обработке связана именно с этим. Есть идея переделать пайплайн и поменять местами анализ бертом и выделение кореференции. Возможно это позволит присвоить их к группам шуток, упомянутым выше. Также в таких шутках упоминалась необходимость проведения синтаксического разбора предложения. 

Встречались и проблемные случаи. Было выявлено две категории:

4) "??????доразобрать с интернетом" - в эту категорию попали шутки, где автор не смог понять какие-либо экстралингвистические нюансы, на которых скорее всего завязан семантический сдвиг, а потому не смог понять щутки в целом. Но есть смысл покапаться на форумах и поискать объяснение.
- Вопрос: насколько это вообще по-научному в таком случае опираться на интерпретацию стороннего человека с форума? Надо ли отбросить эту шутку?

5) "нужен синтакс разбор + онтология. Обсудить шутку" - это шутки, при анализе которых автор в какой-то момент задумался, можно ли тут вообще автоматически отследить сдвиг без онтологии. Было решено собрать такие шутки в отдельную категорию и вынести на обсуждение.


